# Read in feature matrices and class labels and test performance of Extremely random forest ExtraTreesClassifier

Author: Kiran Bhattacharyya

Revisions: 
- 5/11/18 - DRM - translate .py files into .ipynb, misc formatting 
- 5/18/18 - DRM - Implement multicore processing of Extra Trees, use existing sklearn functions in place of newly defined ones


1. loads in feature matrices and class labels from claim and not claim datasets (created with `feature_Preprocessing.ipynb`
2. trains a ExtraTreesClassifier on the entire dataset to do feature selection
3. orders the feature matrix by the order of increasing importance of features
4. iteratively trains extra trees classifiers on increasing subsets of features with cross validation
5. saves results of study as `CrossValidationResults_ExtraTrees_10estimators.pkl`

need to train on subset and leave out a test set (~5%?)


In [49]:
# import relevant libraries
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import time


Set parallel processing for scikitlearn. 
You can set values with positive numbers: 1=1 core, 2=2 cores, etc. or with negative numbers:
-1=all cores, =2, all but one core.

If using a dedicated computer, -1 will be fastest. If wanting to use the computer for other things as well, -2 is best.

In [2]:
cores=-2

define timer code

In [3]:
def stopwatch(start):
    #initialize with start=time.time()
    print(time.time()-start)

### define a function that perform cross validation with train-test splitting on a given dataset

In [34]:
def RandomForestCrossVal(X, y, numCrossVals, testRat):
    Accu_0 = np.zeros(numCrossVals)
    Accu_1 = np.zeros(numCrossVals)
    Accu_a = np.zeros(numCrossVals)
    y = np.ravel(y)

    for i in range(0,numCrossVals):
        clf = ExtraTreesClassifier(n_estimators=10,n_jobs=cores)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRat)
        clf.fit(X_train, y_train)
        myPred_forest = clf.predict(X_test)
        forestConfMat = confusion_matrix(y_test, myPred_forest)
        Accu_0[i] = forestConfMat[0,0]/float(np.sum(forestConfMat[:,0]))
        Accu_1[i] = forestConfMat[1,1]/float(np.sum(forestConfMat[:,1]))
        Accu_a[i] = (forestConfMat[0,0] + forestConfMat[1,1])/float(len(y_test))
# 
    Accu_0_mean = np.mean(Accu_0)
    Accu_0_std = np.std(Accu_0)
    Accu_1_mean = np.mean(Accu_1)
    Accu_1_std = np.std(Accu_1)
    Accu_a_mean = np.mean(Accu_a)
    Accu_a_std = np.std(Accu_a)

    return Accu_0_mean, Accu_0_std, Accu_1_mean, Accu_1_std, Accu_a_mean, Accu_a_std

In [5]:
# load data
allFeatureName = pd.read_pickle('../Data/allFeatureNames.pkl') # get names of features
needCiteFeat = np.load('../Data/NeedCiteFeatMat.npy') # feature matrix for citation needed dataset
notClaimFeat = np.load('../Data/NotClaimFeatMat.npy') # feature matrix for no citation needed (not a claim) dataset

# concatenate all data
allFeats = np.concatenate((needCiteFeat, notClaimFeat), 0)
allClass = np.concatenate((np.ones((len(needCiteFeat), 1)), np.zeros((len(notClaimFeat), 1))), 0)
needCiteFeat = list() # clear data to save memory
notClaimFeat = list()

## Note that class=1 are the samples that need citiations

In [6]:
allFeats

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.]])

Number of samples

In [7]:
len(allFeats)

106693

number of features

In [8]:
len(allFeats[0])

4048

In [9]:
allFeatureName.head(10)

,FeatName
0,wright
1,secretari
2,1937
3,dye
4,begun
5,jitter
6,comic
7,meter
8,groom
9,mere


In [10]:
allFeatureName.tail(10)

,FeatName
4038,CD
4039,RBS
4040,VB
4041,NNPS
4042,DT
4043,PDT
4044,VBP
4045,JJR
4046,NN
4047,VBN


In [11]:
allFeatureName[allFeatureName.FeatName==']']

,FeatName
2823,]


In [12]:
allFeatureName[2820:2825]

,FeatName
2820,hypothes
2821,applic
2822,sniper
2823,]
2824,merger


2823 needs to be removed from the Feature set.

In [13]:
allClass

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

In [14]:
## Original code. Accidentally removes the wrong value
# # feature number 2353 is character ']' and is irrelevant, an error in data pre processing
# allFeats[:,2353] = 0
# allFeats[:,2353]

#update 5/18/18 DRM

allFeats[:,2823] = 0
allFeats[:,2823]

array([0., 0., 0., ..., 0., 0., 0.])

Note: I should do a quick check to make sure that nothing else slipped through pre-processing

In [15]:
# # randomly reorder elements
# reOrder = np.random.permutation(len(allFeats))
# allFeats = allFeats[reOrder,:]
# allClass = allClass[reOrder,:]
# y = np.ravel(allClass)



train_test will randomly shuffle while doing the split. We need to save a split to be completely untouched by the tuning to prevent overfitting.


In [16]:

y = np.ravel(allClass)
X_train, X_test, y_train, y_test = train_test_split(allFeats,y,test_size=.2,random_state=2823)

In [18]:
#because I'm paranoid about it
sum(X_train[:,2823])

0.0

In [19]:
y_train[0]

1.0

### Set up the classification model

DRM: I'll eventually add in some model selection and tuning here.

### perform random forest classification to get feature importances

Edit: can just do a Decision tree classifier to get feature importances. Cuts down on feature selection time by ~ 6x on a single core.
Running both original and DT yields a very close/near identical ordering of feature importances.

In [20]:
# start=time.time()
# clf = ExtraTreesClassifier(n_estimators=100,n_jobs=cores) #set n_jobs to # of cores you want to use
# clf.fit(X_train, y_train)
# featImp = clf.feature_importances_ # get feature importances
# featImpSort = np.argsort(featImp)
# stopwatch(start)

In [21]:
# indices = np.argsort(featImp)[::-1]

# print("Feature ranking:")

# for f in range(allFeats.shape[1]):
#     print("%d. feature %d (%f)" % (f + 1, indices[f], featImp[indices[f]]))


In [22]:
# np.mean(featImp)

In [23]:
start=time.time()
dclf = DecisionTreeClassifier() #set n_jobs to # of cores you want to use
dclf.fit(X_train, y_train)
stopwatch(start)

817.0784568786621


In [24]:
dtfeatImp = dclf.feature_importances_ # get feature importances
dtfeatImpSort = np.argsort(dtfeatImp)
dtindices = np.argsort(dtfeatImp)[::-1]

print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %d ('%s') (%f)" % (f + 1, dtindices[f], allFeatureName.FeatName[dtindices[f]], dtfeatImp[dtindices[f]]))


Feature ranking:
1. feature 4047 ('VBN') (0.012781)
2. feature 4027 ('JJ') (0.012573)
3. feature 4038 ('CD') (0.010494)
4. feature 4036 ('VBG') (0.009328)
5. feature 4040 ('VB') (0.008945)
6. feature 4032 ('NNP') (0.008697)
7. feature 4033 ('NNS') (0.008330)
8. feature 4029 ('VBZ') (0.007844)
9. feature 4046 ('NN') (0.007483)
10. feature 4042 ('DT') (0.007387)
11. feature 4034 ('WDT') (0.006219)
12. feature 4044 ('VBP') (0.006081)
13. feature 4028 ('VBD') (0.005930)
14. feature 3691 ('this') (0.005239)
15. feature 3395 ('1') (0.004871)
16. feature 2470 ('some') (0.004800)
17. feature 4030 ('WRB') (0.004370)
18. feature 3747 ('use') (0.004219)
19. feature 1309 ('includ') (0.003889)
20. feature 3651 ('other') (0.003882)
21. feature 706 ('time') (0.003835)
22. feature 99 ('be') (0.003812)
23. feature 3713 ('has') (0.003786)
24. feature 4041 ('NNPS') (0.003658)
25. feature 4035 ('JJS') (0.003189)
26. feature 3680 ('been') (0.003105)
27. feature 4037 ('RB') (0.002981)
28. feature 968 ('thes

1327. feature 3620 ('quicktim') (0.000195)
1328. feature 2636 ('ge') (0.000195)
1329. feature 2746 ('modul') (0.000195)
1330. feature 1020 ('dilut') (0.000194)
1331. feature 1513 ('page') (0.000194)
1332. feature 3674 ('120') (0.000194)
1333. feature 3705 ('duti') (0.000194)
1334. feature 2602 ('bond') (0.000194)
1335. feature 4007 ('healthi') (0.000194)
1336. feature 3003 ('stem') (0.000194)
1337. feature 3986 ('linux') (0.000193)
1338. feature 2854 ('x') (0.000193)
1339. feature 494 ('jewel') (0.000193)
1340. feature 3319 ('texa') (0.000193)
1341. feature 2423 ('hawk') (0.000192)
1342. feature 2115 ('labrador') (0.000192)
1343. feature 1199 ('catastroph') (0.000192)
1344. feature 2013 ('apostol') (0.000192)
1345. feature 1492 ('medal') (0.000192)
1346. feature 428 ('walt') (0.000192)
1347. feature 303 ('hole') (0.000192)
1348. feature 4003 ('ceremoni') (0.000192)
1349. feature 3927 ('mathematician') (0.000192)
1350. feature 2196 ('cemeteri') (0.000192)
1351. feature 1019 ('eastward')

2327. feature 208 ('multin') (0.000089)
2328. feature 249 ('matur') (0.000089)
2329. feature 3941 ('incorrect') (0.000089)
2330. feature 3022 ('realiti') (0.000088)
2331. feature 1136 ('emulsifi') (0.000088)
2332. feature 2434 ('compost') (0.000088)
2333. feature 1275 ('resurg') (0.000088)
2334. feature 3638 ('guatemala') (0.000088)
2335. feature 1967 ('alga') (0.000088)
2336. feature 3617 ('cohen') (0.000088)
2337. feature 873 ('bark') (0.000088)
2338. feature 1534 ('palestin') (0.000088)
2339. feature 3546 ('fold') (0.000088)
2340. feature 2553 ('kabul') (0.000088)
2341. feature 3785 ('agenda') (0.000088)
2342. feature 3596 ('organiz') (0.000088)
2343. feature 825 ('snow') (0.000088)
2344. feature 865 ('remedi') (0.000088)
2345. feature 3899 ('epic') (0.000088)
2346. feature 667 ('semiconductor') (0.000088)
2347. feature 3521 ('passion') (0.000088)
2348. feature 1878 ('biographi') (0.000088)
2349. feature 1025 ('wrap') (0.000088)
2350. feature 2353 ('firework') (0.000087)
2351. featu

3326. feature 2506 ('frontier') (0.000001)
3327. feature 3299 ('graze') (0.000001)
3328. feature 1211 ('lodg') (0.000001)
3329. feature 3959 ('soap') (0.000001)
3330. feature 535 ('portabl') (0.000001)
3331. feature 855 ('galaxi') (0.000001)
3332. feature 3925 ('buffalo') (0.000001)
3333. feature 82 ('tyrosin') (0.000001)
3334. feature 3547 ('2nd') (0.000001)
3335. feature 2402 ('libertarian') (0.000001)
3336. feature 2851 ('exempt') (0.000001)
3337. feature 603 ('dairi') (0.000001)
3338. feature 3878 ('justif') (0.000001)
3339. feature 2135 ('dispers') (0.000001)
3340. feature 218 ('workforc') (0.000001)
3341. feature 2497 ('hugh') (0.000001)
3342. feature 1263 ('ben') (0.000001)
3343. feature 1613 ('steal') (0.000001)
3344. feature 3921 ('freight') (0.000000)
3345. feature 268 ('deposit') (0.000000)
3346. feature 445 ('1922') (0.000000)
3347. feature 939 ('nose') (0.000000)
3348. feature 2100 ('karl') (0.000000)
3349. feature 2896 ('merchant') (0.000000)
3350. feature 58 ('bah') (0.0

In [25]:
np.mean(dtfeatImp)

0.00024703557312252963

In [26]:
featlim=sum(dtfeatImp>np.mean(dtfeatImp))
featlim

1026

In [48]:
dclf.tree_.max_depth

617

For feature selection, one rule of thumb is you choose features that have greater importance than mean. Beyond that, decision trees are pretty good at feature selection so it's likely not really important to tune based on # of features.
With trees, its better to overfit and then prune with limiting branch depth, or reduce variance by increasing # estimators

### rearrange dimensions so they are sorted by feature importance


In [65]:
# allFeats_sort = np.zeros((len(allFeats), int(allFeats.shape[1])))
# featName_sort = list()
# for i in range(0,len(featImpSort)):
#     indx = featImpSort[-(i + 1)]
#     allFeats_sort[:,i] = allFeats[:,indx]
#     featName_sort.append(allFeatureName.FeatName[indx])


X_train_sort = np.zeros((len(X_train), int(X_train.shape[1])))
featName_sort = list()
for i in range(0,len(dtfeatImpSort)):
    indx = dtfeatImpSort[-(i + 1)]
    X_train_sort[:,i] = X_train[:,indx]
    featName_sort.append(allFeatureName.FeatName[indx])

KeyboardInterrupt: 

In [28]:
featName_sort[0:25]

['VBN',
 'JJ',
 'CD',
 'VBG',
 'VB',
 'NNP',
 'NNS',
 'VBZ',
 'NN',
 'DT',
 'WDT',
 'VBP',
 'VBD',
 'this',
 '1',
 'some',
 'WRB',
 'use',
 'includ',
 'other',
 'time',
 'be',
 'has',
 'NNPS',
 'JJS']

### create subsets of dimensions to use based on importance 100-1000 so it is less than 100 fold of Ns


In [ ]:
testRat = 0.2
numCrossVals = 10
dimsToTry = np.arange(100,1100,100)
dimMeanAccu0 = np.zeros(len(dimsToTry))
dimStdAccu0 = np.zeros(len(dimsToTry))
dimMeanAccu1 = np.zeros(len(dimsToTry))
dimStdAccu1 = np.zeros(len(dimsToTry))
dimMeanAccua = np.zeros(len(dimsToTry))
dimStdAccua = np.zeros(len(dimsToTry))


### perform 10-fold cross validation on each dimension

This is a grid search that can be optimized with sklearn

In [66]:
import time

start = time.time()

for i in range(0,len(dimsToTry)):
    dims = dimsToTry[i]
    thisX = X_train_sort[:,0:dims]
    Accu_0_mean, Accu_0_std, Accu_1_mean, Accu_1_std, Accu_a_mean, Accu_a_std = RandomForestCrossVal(thisX, y_train, numCrossVals, testRat)
    dimMeanAccu0[i] = Accu_0_mean
    dimStdAccu0[i] = Accu_0_std
    dimMeanAccu1[i] = Accu_1_mean
    dimStdAccu1[i] = Accu_1_std
    dimMeanAccua[i] = Accu_a_mean
    dimStdAccua[i] = Accu_a_std
end = time.time()
print(end - start)

1373.9910080432892


In [ ]:

# start = time.time()

# dims = 100
# thisX = allFeats_sort[:,0:dims]
# Accu_0_mean, Accu_0_std, Accu_1_mean, Accu_1_std, Accu_a_mean, Accu_a_std = RandomForestCrossVal(thisX, y, numCrossVals, testRat)
# dimMeanAccu0[i] = Accu_0_mean
# dimStdAccu0[i] = Accu_0_std
# dimMeanAccu1[i] = Accu_1_mean
# dimStdAccu1[i] = Accu_1_std
# dimMeanAccua[i] = Accu_a_mean
# dimStdAccua[i] = Accu_a_std

# end = time.time()
# print(end - start)

In [67]:
dimMeanAccua


array([0.78903403, 0.81303966, 0.82420479, 0.83073634, 0.83528206,
       0.83808213, 0.83990979, 0.84083533, 0.84356511, 0.84278015])

In [ ]:
# start = time.time()

# # for i in range(0,len(dimsToTry)):
# dims=100
# thisX = allFeats_sort[:,0:dims]
# scores = cross_val_score(clf, thisX, y, cv=10,n_jobs=cores)
# end = time.time()
# print(end - start)

In [ ]:
scores

In [ ]:
# start = time.time()

# for i in range(0,len(dimsToTry)):
#     dims = dimsToTry[i]
#     thisX = allFeats_sort[:,0:dims]
#     Accu_0_mean, Accu_0_std, Accu_1_mean, Accu_1_std, Accu_a_mean, Accu_a_std = RandomForestCrossVal(thisX, y, numCrossVals, testRat)
#     dimMeanAccu0[i] = Accu_0_mean
#     dimStdAccu0[i] = Accu_0_std
#     dimMeanAccu1[i] = Accu_1_mean
#     dimStdAccu1[i] = Accu_1_std
#     dimMeanAccua[i] = Accu_a_mean
#     dimStdAccua[i] = Accu_a_std
# stopwatch(start)

In [ ]:
# print(max(dimMeanAccua),list(dimMeanAccua).index(max(dimMeanAccua)))

Instead of optimizing over the number of features, we should optimize over other hyperparameters in the dataset. Number of features will be set to the rule of thumb, features with greater than mean importance. (featlim)

In [110]:
clf1 = ExtraTreesClassifier(n_estimators=10,n_jobs=cores) #extremely random trees does bootstrapping and then averages.

start = time.time()
thisX = X_train_sort[:,0:featlim]
clf1.fit(thisX, y_train)
# scores1 = cross_val_score(clf1, thisX, y_train, cv=10,n_jobs=cores)

stopwatch(start)

43.4507691860199


In [69]:
np.mean(scores1)

0.8574993531143524

In [112]:
clf1b = ExtraTreesClassifier(bootstrap=True,n_jobs=cores) #extremely random trees does bootstrapping and then averages.

start = time.time()
thisX = X_train_sort[:,0:featlim]
clf1b.fit(thisX, y_train)
# scores1b = cross_val_score(clf1b, thisX, y_train, cv=10,n_jobs=cores)

stopwatch(start)

20.950212240219116


In [77]:
np.mean(scores1b)

0.8529300579874997

In [72]:
clf2 = AdaBoostClassifier() #adaboost has weak learners "vote" on the class

start = time.time()
thisX = X_train_sort[:,0:featlim]
scores2 = cross_val_score(clf2, thisX, y_train, cv=10,n_jobs=cores)

stopwatch(start)

664.3448140621185


In [73]:
np.mean(scores2)

0.5845771287442745

In [74]:
clf3 = GradientBoostingClassifier() # gradient boosting trains the next weak learner on misclassified data

start = time.time()
thisX = X_train_sort[:,0:featlim]
scores = cross_val_score(clf3, thisX, y_train, cv=10,n_jobs=cores)

stopwatch(start)

3180.265585899353


In [75]:
scores

array([0.59903948, 0.60039831, 0.60039831, 0.60339777, 0.59906268,
       0.59425893, 0.58886936, 0.59308729, 0.58968951, 0.60093732])

In [85]:

start = time.time()
glf = ExtraTreesClassifier(random_state=1,n_jobs=-1,bootstrap=True)
# TODO: Create the parameters list you wish to tune
parameters = {'max_depth':(60,300,None),'n_estimators':(10,50,100,500)} 

# TODO: Perform grid search on the classifier using 'scorer' as the scoring method
grid_obj = GridSearchCV(glf,parameters,n_jobs=-1,cv=3)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_fit = grid_obj.fit(thisX,y_train)

# Get the estimator
best_clf = grid_fit.best_estimator_

# # Make predictions using the unoptimized and model
# predictions = (glf.fit(thisX, y_train)).predict(X_test)
# best_predictions = best_clf.predict(X_test)


stopwatch(start)

8020.967255830765


In [89]:
grid_fit.best_estimator_

ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=300, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [87]:
grid_fit.best_params_

{'max_depth': 300, 'n_estimators': 500}

In [88]:
grid_fit.best_score_

0.8248939709914005

In [ ]:
clf1b.

maxed out estimators so worth going deeper even.
Warning that this takes a long time to run.

In [96]:

# TODO: Perform grid search on the classifier using 'scorer' as the scoring method
def gridtry(glf,parameters):
    start = time.time()
    grid_obj = GridSearchCV(glf,parameters,n_jobs=-1,cv=4)

    # TODO: Fit the grid search object to the training data and find the optimal parameters
    grid_fit = grid_obj.fit(thisX,y_train)

    return(grid_fit)
    
    stopwatch(start)



In [97]:
glf = ExtraTreesClassifier(random_state=1,n_jobs=-1,bootstrap=True)

parameters = {'max_depth':(150,300,600),'n_estimators':(500, 1000, 2000)} 

grid_fit=gridtry(glf,parameters)

In [98]:
grid_fit.best_params_

{'max_depth': 300, 'n_estimators': 1000}

In [99]:
grid_fit.best_score_

0.8407456006748366

In [100]:
grid_fit.best_estimator_

ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=300, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

Best model has the following properties: max depth: 300, n_estimators: 1000.

## limit the test set and setup for scoring

In [102]:


X_test_sort = np.zeros((len(X_test), int(X_test.shape[1])))
featName_sort = list()
for i in range(0,len(dtfeatImpSort)):
    indx = dtfeatImpSort[-(i + 1)]
    X_test_sort[:,i] = X_test[:,indx]
#     featName_sort.append(allFeatureName.FeatName[indx])

In [103]:
testX = X_test_sort[:,0:featlim]

In [104]:
grid_fit.score(testX,y_test)

0.8725338581939173

In [113]:
clf1b.score(testX,y_test)

0.8639580111532874

In [ ]:
from sklearn.externals import joblib
joblib.dump(grid_fit.best_estimator_, '../Data/1000tree_tuned_extratrees.pkl') 

In [114]:
from sklearn.externals import joblib
joblib.dump(clf1b, '../Data/10tree_tuned_extratrees.pkl') 

['../Data/10tree_tuned_extratrees.pkl']

## save tuned model


In [108]:
# CrossVal_extraTrees_10estimators.to_pickle('../Data/CrossValidationResults_ExtraTrees_10estimators.pkl')
from sklearn.externals import joblib
joblib.dump(grid_fit.best_estimator_, '../Data/1000tree_tuned_extratrees.pkl') 

['../Data/1000tree_tuned_extratrees.pkl']

### create dataframe to save cross validation results


In [ ]:
# CrossVal_extraTrees_10estimators = pd.DataFrame({
#     'NumOfFeatDims': dimsToTry,
#     'NotClaimAccu': dimMeanAccu0,
#     'NotClaimAccuStd': dimStdAccu0,
#     'ClaimAccu': dimMeanAccu1,
#     'ClaimAccuStd': dimStdAccu1,
#     'OverallAccu': dimMeanAccua,
#     'OverallAccuStd': dimStdAccua
# })

In [ ]:
CrossVal_extraTrees_10estimators.OverallAccu